In [19]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/robloxParentsPosts.csv?token=GHSAT0AAAAAAB6VRXEZRAQ2RD6742XV572QZALOSDA')

In [21]:
df.head()

,url,title,author,date,timestamp,text,subreddit,score,upvote ratio
0,https://www.reddit.com/r/RobloxParents/comment...,Inadequate Parental Controls,Howcanthisbe123,2021-03-25,1.616697e+09,"Despite combing the internet, there seems to b...",t5_2m7jy9,11,0.93
1,https://i.redd.it/wahei45uohz41.jpg,"I Saw my kid playing this unholy Roblox game, ...",BilledSauce,2020-05-18,1.589793e+09,NaN,t5_2m7jy9,10,1.00
2,https://www.reddit.com/r/RobloxParents/comment...,Filter curated content,adobogado,2021-12-25,1.640459e+09,Is there anyway to filter to just see curated ...,t5_2m7jy9,7,1.00
3,https://www.reddit.com/r/RobloxParents/comment...,Account Restrictions,cutenfuzzie,2021-05-06,1.620337e+09,"I changed the settings to be restricted, but I...",t5_2m7jy9,5,0.86
4,https://www.reddit.com/r/RobloxParents/comment...,my son(7) was able to evade the ROBLOX chat block,AnotherSpaceNut,2021-07-15,1.626377e+09,So bedtime arrives and my son needs to log off...,t5_2m7jy9,6,1.00


In [22]:
df['text']

0    Despite combing the internet, there seems to b...
1                                                  NaN
2    Is there anyway to filter to just see curated ...
3    I changed the settings to be restricted, but I...
4    So bedtime arrives and my son needs to log off...
Name: text, dtype: object

In [23]:
df['text'].str.count('game').sum()

20.0

In [24]:
text = " ".join(df['text'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [25]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [26]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: curated content with a frequency of 4


In [31]:
df_no_na = df.dropna(subset=['text'])
df_ques = df_no_na['text'].str.endswith('?')
df_true = df_no_na.loc[df_ques]
print(df_true['text'])

2    Is there anyway to filter to just see curated ...
3    I changed the settings to be restricted, but I...
Name: text, dtype: object


In [32]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                   term  rank
135    curated content   4.0
21     age appropriate   3.0
94          child play   3.0
335  parental controls   3.0
213  generated content   3.0
495     user generated   3.0
266     little nothing   2.0


In [33]:
df['text_processed'] = df['text'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()

0    despite combing the internet there seems to be...
1                                                  NaN
2    is there anyway to filter to just see curated ...
3    i changed the settings to be restricted but i ...
4    so bedtime arrives and my son needs to log off...
Name: text_processed, dtype: object

In [34]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['despite', 'combing', 'internet', 'seems', 'mechanism', 'either', 'block', 'games', 'feel', 'inappropriate', 'child', 'given', 'age', 'go', 'limited', 'curated', 'content', 'child', 'prohibited', 'chatting', 'friends', 'game', 'whatsoever', 'seems', 'entirely', 'unreasonable', 'setting', 'little', 'nothing', 'allow']


In [35]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
